In [56]:
# read data
import pandas as pd
import numpy as np
from IPython.display import clear_output
results = pd.read_csv('results.csv', encoding= 'unicode_escape')

In [57]:
results[results.Season == '2008-09']

,Season,DateTime,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HC,AC,HF,AF,HY,AY,HR,AR
5864,2008-09,2008-08-16T00:00:00Z,Arsenal,West Brom,1,0,H,1.0,0.0,H,H Webb,24.0,5.0,14.0,4.0,7.0,5.0,11.0,8.0,0.0,0.0,0.0,0.0
5865,2008-09,2008-08-16T00:00:00Z,Bolton,Stoke,3,1,H,3.0,0.0,H,C Foy,14.0,8.0,8.0,2.0,4.0,3.0,13.0,12.0,1.0,2.0,0.0,0.0
5866,2008-09,2008-08-16T00:00:00Z,Everton,Blackburn,2,3,A,1.0,1.0,D,A Marriner,10.0,15.0,5.0,11.0,3.0,5.0,11.0,9.0,2.0,2.0,0.0,0.0
5867,2008-09,2008-08-16T00:00:00Z,Hull,Fulham,2,1,H,1.0,1.0,D,P Walton,11.0,12.0,6.0,6.0,5.0,6.0,10.0,9.0,3.0,0.0,0.0,0.0
5868,2008-09,2008-08-16T00:00:00Z,Middlesbrough,Tottenham,2,1,H,0.0,0.0,D,M Atkinson,14.0,8.0,10.0,5.0,7.0,9.0,11.0,12.0,1.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6239,2008-09,2009-05-24T00:00:00Z,Liverpool,Tottenham,3,1,H,1.0,0.0,H,P Walton,20.0,8.0,9.0,4.0,9.0,2.0,14.0,4.0,0.0,1.0,0.0,0.0
6240,2008-09,2009-05-24T00:00:00Z,Man City,Bolton,1,0,H,1.0,0.0,H,M Clattenburg,20.0,8.0,12.0,4.0,15.0,9.0,8.0,10.0,0.0,0.0,0.0,0.0
6241,2008-09,2009-05-24T00:00:00Z,Sunderland,Chelsea,2,3,A,0.0,0.0,D,M Halsey,8.0,14.0,7.0,9.0,6.0,7.0,8.0,7.0,1.0,1.0,0.0,0.0
6242,2008-09,2009-05-24T00:00:00Z,West Ham,Middlesbrough,2,1,H,1.0,0.0,H,H Webb,14.0,17.0,8.0,6.0,4.0,5.0,14.0,10.0,0.0,1.0,0.0,0.0


In [58]:
start = '2010-11'
average_by = 10
weights = [1] * average_by
useless_heads = ['Season', 'DateTime', 'HomeTeam', 'AwayTeam', 'FTR', 'HTR', 'Referee']


In [128]:
def generate_heads():
    heads = np.array(results.columns)
    good_heads = [i for i in heads if i not in useless_heads]
    a_heads = ["a_" + i for i in good_heads]
    b_heads = ["b_" + i for i in good_heads]
    heads = a_heads + b_heads
    return heads

In [130]:
def calculate_means(team_name, label, average_by, index):
    heads = np.array(results.columns)
    good_heads = [i for i in heads if i not in useless_heads]
    res = np.array([0] * len(good_heads))
    cnt = 0
    for index, row in results.loc[index-1:0:-1].iterrows():
        if label == 'A':
            if row['AwayTeam'] == team_name:
                res = res + row[good_heads].to_numpy()
                cnt += 1
        elif label == 'H':
            if row['HomeTeam'] == team_name:
                res = res + row[good_heads].to_numpy()
                cnt += 1
        if cnt == average_by:
            break
    if cnt != average_by:
        return [np.nan] * len(good_heads)
    res /= average_by
    return res
                

In [203]:
def process_data(start, average_by, weights):
    row_list = []
    y = np.array([])
    begin = results[results.Season == start].index[0]
    for index, row in results.loc[begin:].iterrows():
        a_row = calculate_means(row['HomeTeam'], 'H', average_by, index)
        b_row = calculate_means(row['AwayTeam'], 'A', average_by, index)
        new_row = np.append(a_row, b_row)
        if np.isnan(new_row.astype(np.float)).any():
            continue
        row_list.append(new_row)
        #print(row_list)
        y = np.append(y, row['FTR'])
        print(index - begin, "/", results.shape[0] - begin)
        clear_output(wait=True)
    print("Finished")
    X = pd.DataFrame(row_list, columns = generate_heads())
    return X, y

In [204]:
X, y = process_data(start, average_by, weights)
#print(X)

Finished


In [205]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [206]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train)
y_predict = RFC.predict(X_test)

In [209]:
from sklearn.metrics import f1_score
score = f1_score(y_test, y_predict, average="micro")
print(score)

0.5290482076637825
